<a href="https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/20_workbook/ProgramWorkBook_ImageClassfication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# プログラム記述練習：画像分類編


まずはこのプログラムをコピーして自分のGoogleドライブへと保存しましょう．






## 0. データのダウンロード

プログラムに必要なデータをダウンロードします．

**今回使用するデータは，今までと同様にMNISTデータセットですが，異なるデータの構造となっています．今回のデータのファイル構造は実際のデータ収集を行い，学習や評価に使用するために整理されたデータを想定して作成しています．**

In [ ]:
import gdown
gdown.download('https://drive.google.com/uc?id=1dlrHQ0bqgs98q-gYoxJbI4YqUp0brNuA', 'MNIST_dataset.zip', quiet=False)
!unzip -q -o MNIST_dataset.zip

ここで，一度データセットを確認してみましょう．

データを確認すると，MNIST_datasetのフォルダ名trainとtestフォルダがあり，その中に0 ~ 9のフォルダがあります．

それぞれの0 ~ 9のフォルダの中には，各クラスに対応した画像データがpng画像として1枚ずつ保存されています．

![mnistfolder.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/143078/d76cc9e1-a36c-8065-51b7-8e55c385b751.png)

**注意点：オリジナルのMNIST Datsetについて**

オリジナルのデータセットでは，1枚ずつの画像ではなく，バイナリデータとして画像データがまとめて保存されています．
これまでに使用してきた`torchvision.datasets.MNIST`では，このバイナリデータを自動的にダウンロードして，読み込むことでデータセットを準備しています．

MNIST datasetのWebサイト：
http://yann.lecun.com/exdb/mnist/

![MNIST original.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/143078/b85f2d0a-1120-438f-c5e3-21e2e2598450.png)

## 1. モジュールのインポート部分

こちらにプログラムに必要なモジュールのインポートを記述しましょう．

※ プログラムが一通り完成した後に記述します．

## 2. データセットクラスの作成

ここでは，ダウンロードしたMNISTデータセットの形式に合わせて，PyTorchのデータセットクラスを自作します．


### この部分で使用する関数・クラスの動作確認

In [ ]:
import os
import glob
import numpy as np
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

### パスの作成 (os.path.join)
_path = os.path.join("./directory", "sub_directory/subsub_directory", "file")
print("os.path.joinの結果:", _path, "\n--------------------")

### ファイル一覧の取得 (glob)
_glob_result1 = glob.glob("./MNIST_dataset/*")
print("globの結果1: ", _glob_result1, "\n--------------------")

_glob_result2 = glob.glob("./MNIST_dataset/train/*/*.png")
print("globの結果2:", _glob_result2, "\n--------------------")

### 文字列の分割 (split)
_filename = "./MNIST_dataset/train/4/30167.png"
_split_filename = _filename.split("/")
print("文字列の分割結果:", _split_filename, "\n--------------------")

### リストのIndexing（要素選択）
_selected_split_filename = _split_filename[-2]  # 後ろから2番目
print("リストのIndexingの結果:", _selected_split_filename, "\n--------------------")

### 文字列 (str) --> 整数型 (int) への変換
_int_data = int(_selected_split_filename)
print("整数型への変換結果:", _int_data)

### 画像の読み込み (Python Image Library; PIL)
_sample_image = Image.open("./MNIST_dataset/train/4/30167.png")
print(_sample_image, "\n--------------------")   # 注意：画像は表示されません

### PILで読み込んだ画像データの配列形式変換
# 1. PIL Image --> Numpy array (今回は使いませんが頻繁に使用します)
_numpy_image = np.array(_sample_image)
print("image data as numpy array:", _numpy_image, "\n--------------------")

# 2. PIL --> PyTorch Tensor (torchvisionの機能．画素値を自動的に0~255 --> 0.0 ~ 1.0に正規化しつつ変換します．)
_to_tensor_class = ToTensor()
_tensor_image = _to_tensor_class(_sample_image)
print("image data as torch tensor:", _tensor_image, "\n--------------------")

上で確認した機能を使いつつ，データセットクラスを定義します．

### データセットクラスのデバッグ

ここで，一度作成したデータセットクラスを呼び出して実行することで，正しく動作しているかを確認（デバッグ）します．

**※ 細かなデバッグは非常に重要です．全てのプログラムを記述してからデバッグをしようとすると，原因の特定に時間がかかります．一つのクラス・関数などを作成したら，簡単なプログラムを記述してうまく動作するか確認をすることをお勧めします．**

## 3. ネットワークモデルの定義

続いてネットワークを定義します．
畳み込みニューラルネットワークを定義します．

### この部分で使用する関数・クラスの動作確認

In [ ]:
import torch
import torch.nn as nn

# Tensor配列のサイズ確認
x = torch.randn(10, 32, 7, 7)
print("size of original x:", x.size())

# Tensor配列の並べ替え（サイズ変更）
x_dst_1 = x.view(10, 32, 7*7)
x_dst_2 = x.view(-1, 32*7*7)
print("size of x_dst_1:", x_dst_1.shape)
print("size of x_dst_2:", x_dst_2.shape)

上で確認した機能を使いつつ，ネットワーククラスを定義します．

この時，各レイヤーのクラスの詳細などを調べたい場合は，PyTorchのReferenceを参照しつつ実装します．

[PyTorch reference mannual](https://pytorch.org/docs/stable/index.html)

### ネットワーククラスのデバッグ

ここで，一度作成したネットワーククラスを呼び出して実行することで，正しく動作しているかを確認（デバッグ）します．

## 4. 学習の準備

ここでは，学習に必要な

* ネットワークモデル
* 誤差関数
* 最適化手法
* データセット

の定義を行います．

最適化関数などもReference mannualを参照しつつ好きなものを選択記述しましょう．

**DataLoaderのnum_workersについて**

`torch.utils.data.DataLoader`の引数である`num_workers`は，データを読み込んで準備する処理を並列処理するための引数です．例えば，`num_workers=10`とした場合には，10並列でデータの読込処理 (データセットクラスの`__getitem__()`) を10並列で実行してくれます．そのため，使用する計算機のCPU性能に合わせて，ある程度大きな数を指定しておくとデータの読込処理が早くなり，学習の高速化が期待できます．

## 5. 学習の開始

上で定義したモデルや最適化手法，データセットを用いて学習を行います．

### この部分で使用する関数・クラスの動作確認

In [ ]:
### PyTorchのTensor (配列) 操作
_output_sample = torch.rand([2, 10], dtype=torch.float32)
print("network output (example):", _output_sample)

### argmax (最大値の要素をもつ配列のインデックスを返す)
_predicted_class = _output_sample.argmax(dim=1)  # 1-次元目方向（横方向）にargmax
print("predicted class (example):", _predicted_class)

### item (tensorのとある一つの要素をscalerとして返す)
_sample_tensor = torch.tensor([1,2,3,4], dtype=torch.float32)
print("return as scaler:", _sample_tensor[0].item())

### 上のセルで定義したネットワークモデルのパラメータ（+そのほか）の抽出
print(list(model.parameters()))   # 学習するモデルパラメータ（optimizerへ入力する時に使用することが多い）
print("----------------")
print(model.state_dict())         # 学習するパラメータ + 学習はしないけど学習や推論処理を行うことで変化するパラメータ
                                  #  (BatchNormの内部パラメータなど)（モデルの保存などに使用されることが多い）
print("----------------")

### PyTorchのtensorのファイル保存
_save_data = torch.tensor([1,2,3,4], dtype=torch.float32)
torch.save(_save_data, "sample_data.pt")

### PyTorchのtensorのファイルの読込
_load_data = torch.load("sample_data.pt")
print(_load_data)

上記の関数を活用して学習ループを記述します．

### 学習経過のテキスト保存とグラフ化

**テキスト保存**

ログ（コンソール・ターミナル部分）に学習中の誤差の推移などを表示することで，学習の様子を確認できますが，Colaboratoryの再起動や再度学習を行うことでログは消えてしまい．後程，学習経過を確認・比較することができなくなります．
そのため，学習経過の数値をテキストファイルなどで保存しておくと，非常に便利です．
異なるネットワークや学習パラメータを用いて学習を行った場合との比較などを行うことができます．



**グラフ化**

実行結果の数値を見ることで学習の様子が確認できますが，グラフ化することでより直感的に学習の挙動を理解することができます．
ここでは，上の学習プログラム中で保存しておいた，1 epochごとの誤差とテストデータに対する精度をグラフ化して確認してみます．

## 6. 評価

学習したモデルを用いて評価を行います．

### モデルの読み込み

学習中に保存したモデルを読み込みます．



読み込んだ学習済みモデルでテストデータを認識して，認識率を算出します．